In [19]:
from pymed import PubMed
import pandas as pd

pubmed = PubMed(tool="MyTool", email="b.mascat@gmail.com")
farmacos = pd.read_csv("farmacos.csv", header=None, names=['Farmaco'])
tumores = pd.read_csv("tipo_tumores.csv", header=None, names=['TipoTumor'])

# Función para obtener el número de resultados
def get_results_count(query):
    return pubmed.getTotalResultsCount(query)

# Función para procesar una única combinación de tipos de tumor y fármacos
def process_combination(tipo_tumor, farmaco):
    query = f"{tipo_tumor} AND {farmaco}"
    results_count = get_results_count(query)
    return {'TipoTumor': tipo_tumor, 'Farmaco': farmaco, 'Resultados': results_count}


In [20]:
# Inicializar una lista para almacenar los resultados
results_list = []

# Bucle anidado para combinar tipos de tumor y fármacos
for tipo_tumor in tumores['TipoTumor']:
    for farmaco in farmacos['Farmaco']:
        # Crear la consulta combinando el tipo de tumor y el fármaco
        query = f"{tipo_tumor} AND {farmaco}"
        # Obtener el número de resultados
        results_count = get_results_count(query)
        # Agregar los resultados a la lista
        results_list.append({'TipoTumor': tipo_tumor, 'Farmaco': farmaco, 'Resultados': results_count})

# Crear un DataFrame a partir de la lista de resultados
results_df = pd.DataFrame(results_list)

# Mostrar el DataFrame final
print("DataFrame de Resultados:")
print(results_df)

DataFrame de Resultados:
                          TipoTumor                Farmaco  Resultados
0            Gastric Adenocarcinoma          PEMBROlizumab         129
1            Gastric Adenocarcinoma            OSIMERTINIB           4
2            Gastric Adenocarcinoma              NIVOlumab         133
3            Gastric Adenocarcinoma            ABEMACICLIB           0
4            Gastric Adenocarcinoma  EMTANSINA TRASTUZUMAB           0
...                             ...                    ...         ...
2595  Colorectal Carcinoma Invasive              RUCAPARIB           0
2596  Colorectal Carcinoma Invasive               ALBÚMINA           0
2597  Colorectal Carcinoma Invasive           TEMOZOLOMIDA           4
2598  Colorectal Carcinoma Invasive           OXALIPLATINO         355
2599  Colorectal Carcinoma Invasive              SOTORASIB           2

[2600 rows x 3 columns]
